In [1]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np
import dask
from dask.distributed import Client, progress
import pandas as pd

cell = "jrc_hela-2"

client = Client(threads_per_worker=2, n_workers=1)
client.cluster.scale(8)
mp = MeshProcessor(
    path=f"https://janelia-cosem-datasets.s3.amazonaws.com/{cell}/neuroglancer/mesh/mito_seg",
    lod=2,
    min_branch_length=100,
    use_skeletons=True,
)
lazy_results = []
for i in range(1, 422, 1):  # range(1,87077,100): #
    lazy_results.append(mp.process_mesh(id=i))
results = dask.compute(*lazy_results)
df = pd.DataFrame.from_records(results)
temp = df[["pic_0", "pic_1", "pic_2"]].to_numpy() >= 0
print(temp.all())


/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-lqbyv9yr', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-shuqbjwd', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-vl20n6m_', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Programming/mesh_playground/dask-worker-space/worker-xcbmnek6', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/ackermand/Progr

True


In [2]:
from __future__ import print_function

import argparse
from functools import partial
import webbrowser

import neuroglancer
import neuroglancer.cli
import copy
import requests

from IPython.display import IFrame
import numpy as np

#ap = argparse.ArgumentParser()
#neuroglancer.cli.add_server_arguments(ap)
#args = ap.parse_args()
#neuroglancer.cli.handle_server_arguments(args)


viewer = neuroglancer.Viewer()
response = requests.get(f"https://janelia-cosem-datasets.s3.amazonaws.com/{cell}/neuroglancer/mesh/mito_seg/segment_properties/info")
info_file = response.json()
segment_ids = info_file["inline"]["ids"]
mesh_id_to_index_dict = {}
with viewer.txn() as s:
    s.layers["mesh"] = neuroglancer.SegmentationLayer(  # Single MEsh Layer?
        source=f"precomputed://s3://janelia-cosem-datasets/{cell}/neuroglancer/mesh/mito_seg",
    )
    for index,segment_id in enumerate(segment_ids):
        mesh_id_to_index_dict[int(segment_id)] = index
        s.layers["mesh"].segments.add(segment_id)
    s.layout = '3d'

def manually_label(class_key, color, s):

    if s.selected_values["mesh"].value:
        selected_mesh_ids = s.selected_values["mesh"].value
        if s.selected_values["mesh"].value:
            new_state = copy.deepcopy(viewer.state)
            new_state.layers['mesh'].segments.remove(selected_mesh_ids)
            class_layer = f'class {class_key}'
            if class_layer not in new_state.layers: #should do it with name since this is a whole object
                new_state.layers[class_layer] = neuroglancer.SegmentationLayer(  # Single MEsh Layer?
                    source=f"precomputed://s3://janelia-cosem-datasets/{cell}/neuroglancer/mesh/mito_seg")
            new_state.layers[class_layer].segments.add(selected_mesh_ids)

            segment_colors = {}
            for segment_id in new_state.layers[class_layer].segments:
                segment_colors[segment_id] = color
            new_state.layers[class_layer].segment_colors = segment_colors
            
            viewer.set_state(new_state)

def fit_and_predict(s):
    class_layers = ["class h", "class j", "class k", "class l"]
    mesh_index_to_class_dict = {}
    state = viewer.state
    for layer in state.layers:
        if layer.name != "mesh": #then is a class layer
            for segment_id in layer.segments:
                class_id = class_layers.index(layer.name)
                mesh_index = mesh_id_to_index_dict[segment_id]
                mesh_index_to_class_dict[mesh_index] = class_id

viewer.actions.add("my-action-h", partial(manually_label,"h","red"))
viewer.actions.add("my-action-j", partial(manually_label,"j","gray"))
viewer.actions.add("my-action-k", partial(manually_label,"k","blue"))
viewer.actions.add("my-action-l", partial(manually_label,"l","magenta"))

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer["keyh"] = "my-action-h"
    s.input_event_bindings.viewer["keyj"] = "my-action-j"
    s.input_event_bindings.viewer["keyk"] = "my-action-k"
    s.input_event_bindings.viewer["keyl"] = "my-action-l"
    s.input_event_bindings.viewer["keyp"] = "my-action-p"



url = viewer
display(IFrame(url, width=1200, height=800))

https://neuroglancer-demo.appspot.com#!%7B%22layers%22:%5B%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22precomputed://s3://janelia-cosem-datasets/jrc_hela-2/neuroglancer/mesh/mito_seg%22%7D%5D,%22segments%22:%5B%22258%22,%22139%22,%22230%22,%2274%22,%22274%22,%22150%22,%22109%22,%22116%22,%22399%22,%22334%22,%22335%22,%2226%22,%2290%22,%22117%22,%22387%22,%22177%22,%22412%22,%22417%22,%22369%22,%2214%22,%223%22,%2285%22,%22337%22,%2289%22,%22125%22,%22100%22,%22272%22,%22276%22,%2294%22,%2282%22,%222%22,%229%22,%22175%22,%22234%22,%22301%22,%22238%22,%22362%22,%2227%22,%2236%22,%22191%22,%22365%22,%2223%22,%22133%22,%22419%22,%22371%22,%22381%22,%22138%22,%22311%22,%22366%22,%22279%22,%22189%22,%2212%22,%22222%22,%22418%22,%22102%22,%22101%22,%22224%22,%22256%22,%22389%22,%22401%22,%22156%22,%2210%22,%22148%22,%22357%22,%22308%22,%22190%22,%22107%22,%22290%22,%22243%22,%22264%22,%22355%22,%2291%22,%2255%22,%22263%22,%22203%22,%22111%22,%22392%22,%22197%22,%22348%22,%

In [3]:
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

class FitAndPredict():
    def __init__(self, df, metrics):
        self.metrics = df[metrics].to_numpy()
    
    def fit(self, mesh_index_to_class_dict):
        self.classifier = MLPClassifier(alpha=1, max_iter=1000)
        classes = list(mesh_index_to_class_dict.values())

        mesh_indices = list(mesh_index_to_class_dict.keys())
        data = self.metrics[mesh_indices, :]
        self.scaler = preprocessing.StandardScaler().fit(data)
        data_transformed = self.scaler.transform(data)
        self.classifier.fit(data_transformed, classes)
        print(*zip(data_transformed,classes))
    def predict(self):
        test_transformed = self.scaler.transform(self.metrics)
        class_predictions = self.classifier.predict(test_transformed)
        return class_predictions

viewer.actions.remove("my-action-p",fit_and_predict)
def fit_and_predict(s):
    class_layers = ["class h", "class j", "class k", "class l"]
    mesh_index_to_class_dict = {}
    state = viewer.state
    for layer in state.layers:
        if layer.name != "mesh": #then is a class layer
            for segment_id in layer.segments:
                class_id = class_layers.index(layer.name)
                mesh_index = mesh_id_to_index_dict[segment_id]
                mesh_index_to_class_dict[mesh_index] = class_id
    
    fp = FitAndPredict(df, ["ob_normalized_0","ob_normalized_1","ob_normalized_2","pic_normalized_0","pic_normalized_1","pic_normalized_2","longest_path","num_fragments"])
    fp.fit(mesh_index_to_class_dict)
    class_predictions = fp.predict()

    new_state = copy.deepcopy(viewer.state)
    mesh_layer = new_state.layers['mesh']

    colors = ["red", "gray", "blue", "magenta"]
    
    segment_colors = {}
    for segment_id in mesh_layer.segments:
        mesh_index = mesh_id_to_index_dict[int(segment_id)]
        class_id = class_predictions[mesh_index]
        segment_colors[segment_id] = colors[class_id]

    mesh_layer.segment_colors = segment_colors
    viewer.set_state(new_state)

viewer.actions.add("my-action-p", fit_and_predict)

(array([ 1.20472313, -0.30943341, -1.13793067,  0.73057082,  0.59845178,
       -1.26604446, -0.69564456, -0.5       ]), 0) (array([-1.62681793,  0.40485268,  1.54161346, -0.95131611, -0.74661411,
        1.62119871,  1.36987495, -0.5       ]), 0) (array([-0.43701608, -0.84602919,  0.78072283, -1.32799499,  1.11417732,
        0.45501426,  1.06258172,  2.        ]), 0) (array([ 0.84312937, -0.99269948, -0.4983805 ,  0.22738065,  0.60704738,
       -0.74686024, -0.90144531, -0.5       ]), 1) (array([ 0.01598151,  1.74330939, -0.68602513,  1.32135964, -1.57306237,
       -0.06330827, -0.83536681, -0.5       ]), 1)
